In [ ]:
-- CREATE COMPUTE POOL MAHDI_GPU_COMPUTE_POOL
--   MIN_NODES = 1
--   MAX_NODES = 1
--   INSTANCE_FAMILY = GPU_NV_M;
-- CREATE OR REPLACE IMAGE REPOSITORY Qwen3_30B_A3B_image_repo;


In [ ]:
from snowflake.ml.utils import connection_params
from snowflake.snowpark import Session
from snowflake import snowpark
from snowflake.ml.registry import registry as registry_module
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions
import huggingface_hub
from snowflake import snowpark

In [1]:
! pip install huggingface_hub==0.34

In [ ]:
!pip install --upgrade transformers

In [ ]:
!pip install openai

In [2]:
model = "Qwen/Qwen3-30B-A3B"
cache_dir = "Qwen/Qwen3_30B_A3B"

repo_snapshot_dir = huggingface_hub.snapshot_download(
    repo_id=model,
    cache_dir=cache_dir,
    # token=token,
    # allow_patterns=allow_patterns,
    # ignore_patterns=ignore_patterns,
)
repo_snapshot_dir

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

'qwen_2_dot_5_0_dot_5b/models--Qwen--Qwen2.5-0.5B/snapshots/060db6499f32faf8b98477b0a26969ef7d8b9987'

In [ ]:
# from snowflake.snowpark.context import get_active_session

# # Get the current session
# session = get_active_session()

# # Assuming 'myfile.csv' was created in the notebook's temporary filesystem
# # And 'my_permanent_stage' is a stage you've created
# put_result = session.file.put(
#     'Qwen/Qwen3_30B_A3B/models--Qwen--Qwen3-30B-A3B/snapshots/ad44e777bcd18fa416d9da3bd8f70d33ebb85d39/*', # Local path in notebook's temporary filesystem
#     '@MODELSTAGE/q3/', # Target stage path
#     overwrite=True
# )
# print(put_result)

In [3]:
from snowflake.ml.model import HuggingFacePipelineModel
model_ref = HuggingFacePipelineModel(
    model="Qwen/Qwen3-30B-A3B",
    task="text-generation",
    download_snapshot=False,
)
assert model_ref.repo_snapshot_dir is None
model_ref.repo_snapshot_dir = repo_snapshot_dir
assert not model_ref.repo_snapshot_dir is None

/opt/anaconda3/envs/snowml/lib/python3.10/site-packages/snowflake/connector/options.py:104: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.0), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(
/opt/anaconda3/envs/snowml/lib/python3.10/site-packages/snowflake/snowpark/session.py:36: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [4]:
session = session = get_active_session()
assert session is not None
registry = registry_module.Registry(
    session=session,
)
assert registry is not None

SnowflakeLoginOptions() is deprecated since 1.8.5. 


In [5]:
from snowflake.ml.model import openai_signatures

model_name = "Qwen3_30B_A3B_model_final"
mv = registry.log_model(
    model=model_ref,
    model_name=model_name,
    signatures=openai_signatures.OPENAI_CHAT_SIGNATURE
)
mv

Logging model: creating model manifest...:  33%|███▎      | 2/6 [00:02<00:04,  1.16s/it]  

/opt/anaconda3/envs/snowml/lib/python3.10/site-packages/snowflake/ml/registry/_manager/model_parameter_reconciler.py:72: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  reconciled_options = self._reconcile_relax_version(reconciled_options, reconciled_target_platforms)
/opt/anaconda3/envs/snowml/lib/python3.10/site-packages/snowflake/ml/model/_packager/model_packager.py:90: UserWarning: It is impossible to validate your model signatures when using a `snowflake.ml.model.models.huggingface_pipeline.HuggingFacePipelineModel` object. Please make sure you are providing correct model signatures.
  handler.save_model(


Model logged successfully.: 100%|██████████| 6/6 [05:08<00:00, 51.44s/it]                          


ModelVersion(
  name='QWEN_2_DOT_5_0_DOT_5B_MODEL',
  version='PLASTIC_FOX_4',
)

In [ ]:
from snowflake.ml.utils import connection_params
from snowflake.snowpark import Session
from snowflake import snowpark
from snowflake.ml.registry import registry as registry_module
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

session = session = get_active_session()
assert session is not None
registry = registry_module.Registry(
    session=session,
)
assert registry is not None
model_name = "Qwen3_30B_A3B_model_final"
version_name = "WICKED_BADGER_1"
mv = registry.get_model(model_name).version(version_name)
from snowflake.ml.model.inference_engine import InferenceEngine

mv.create_service(
    service_name="Qwen3_30B_A3B_service_vLLM",    
    service_compute_pool="MAHDI_GPU_COMPUTE_POOL",  # Pick a compute pool name with GPU
    gpu_requests="4",  # 4 if the compute pool has 4 GPUs
    ingress_enabled=True,  # set a False if you don't want to create a service endpoint
    image_repo="MAHDI_PLAYGROUND.LLMS.Qwen3_30B_A3B_image_repo",  # Optional: specify a container services image repo name
    experimental_options={
        "inference_engine": InferenceEngine.VLLM,
        "inference_engine_args_override": [
             "--tensor-parallel-size=4",
             "--gpu-memory-utilization=0.85",
             "--max-model-len=1024",
             "--max-num-seqs=2"
        ],
    },)

In [ ]:
session = session = get_active_session()
assert session is not None
registry = registry_module.Registry(
    session=session,
)
assert registry is not None
model_name = "Qwen3_30B_A3B_model_final"
version_name = "WICKED_BADGER_1"
mv = registry.get_model(model_name).version(version_name)
services_df = mv.list_services()
endpoint = services_df.iloc[-1]["inference_endpoint"]
url = f"https://{endpoint}/{version_name}"
url
services_df 

In [ ]:
import openai
client = openai.OpenAI(base_url=url, api_key="not_a_real_key")

In [ ]:
import openai
client = openai.OpenAI(base_url=url, api_key="not_a_real_key")

pat = "...."
model_name="QWEN3_30B_A3B_MODEL_FINAL"

messages = [
    {"role": "system", "content": "You are a helpful assistant. Answer in a concise manner."},
    {"role": "user", "content": "Who is your creator?"},
]


completions = client.chat.completions.create(
    model=model_name,
    messages=messages,
    extra_headers={"Authorization": f'Snowflake Token="{pat}"'},
    max_completion_tokens=250,
)
completions

In [ ]:
import json
input_dataframe=session.sql("""
SELECT
        array_construct(
            object_construct(
                'role',
                'system',
                'content',
                'You are Qwen, created by Alibaba Cloud. You are a helpful assistant.'
            ),
            object_construct(
                'role',
                'user',
                'content',
                'What is the capital of Italy?'
            ))  "MESSAGES",
            '0.5'::FLOAT  "TEMPERATURE",
            512::INT  "MAX_COMPLETION_TOKENS",            
            NULL::ARRAY  "STOP",
            1::INT  "N",
            False::BOOLEAN  "STREAM",
            '1.0'::FLOAT  "TOP_P",
            '0.1'::FLOAT  "FREQUENCY_PENALTY",
            '0.1'::FLOAT  "PRESENCE_PENALTY"
            """)
from snowflake.ml.registry import registry
reg = registry.Registry(session=session, database_name='MAHDI_PLAYGROUND', schema_name='PUBLIC')
mv = reg.get_model('Qwen3_30B_A3B_model_final').version('WICKED_BADGER_1')
completion=mv.run(input_dataframe,service_name="QWEN3_30B_A3B_SERVICE_VLLM")

completion_pandas = completion.to_pandas()
choices_string= completion_pandas["CHOICES"][0]
json_object = json.loads(choices_string)
message_content = json_object[0]["message"]["content"]
# finish_reason = json_object["finish_reason"]
print(message_content)